# King County

In [45]:
DATASET_VERSION="a"
DATASET_VERSIONS=[f"{DATASET_VERSION}-{i+1}" for i in range(3)]
DATASET_NAME = "King County"
import pandas as pd
import numpy as np

from syntheticml.data.synthetic import Synthetic, MODELS
df = pd.read_csv('../datasets/kingcounty/raw/kc_house_data.csv');


In [46]:
def get_syn(data_version):
    syn = Synthetic(df,
                    id="id",
                    category_columns=("condition", "floors", "grade", "view",
                                      "waterfront", "zipcode", "bathrooms", "bedrooms",),
                    synthetic_folder=f"../datasets/kingcounty/synth-{data_version}",
                    models=MODELS.keys(),
                    n_sample=21613,
                    max_cpu_pool=1,
                    target_column="price"
                    )
    syn.process()
    syn.process_scores()
    return syn

In [47]:
syns = [get_syn(data_version) for data_version in DATASET_VERSIONS]


{'columns': {'id': {'sdtype': 'numerical'}, 'date': {'sdtype': 'categorical'}, 'price': {'sdtype': 'numerical'}, 'bedrooms': {'sdtype': 'categorical'}, 'bathrooms': {'sdtype': 'categorical'}, 'sqft_living': {'sdtype': 'numerical'}, 'sqft_lot': {'sdtype': 'numerical'}, 'floors': {'sdtype': 'categorical'}, 'waterfront': {'sdtype': 'categorical'}, 'view': {'sdtype': 'categorical'}, 'condition': {'sdtype': 'categorical'}, 'grade': {'sdtype': 'categorical'}, 'sqft_above': {'sdtype': 'numerical'}, 'sqft_basement': {'sdtype': 'numerical'}, 'yr_built': {'sdtype': 'numerical'}, 'yr_renovated': {'sdtype': 'numerical'}, 'zipcode': {'sdtype': 'categorical'}, 'lat': {'sdtype': 'numerical'}, 'long': {'sdtype': 'numerical'}, 'sqft_living15': {'sdtype': 'numerical'}, 'sqft_lot15': {'sdtype': 'numerical'}}, 'METADATA_SPEC_VERSION': 'SINGLE_TABLE_V1'}
Fitting Models
Params
{'metadata': {
    "columns": {
        "id": {
            "sdtype": "numerical"
        },
        "date": {
            "sdtype":

/usr/local/python/3.10.13/lib/python3.10/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(


{'columns': {'id': {'sdtype': 'numerical'}, 'date': {'sdtype': 'categorical'}, 'price': {'sdtype': 'numerical'}, 'bedrooms': {'sdtype': 'categorical'}, 'bathrooms': {'sdtype': 'categorical'}, 'sqft_living': {'sdtype': 'numerical'}, 'sqft_lot': {'sdtype': 'numerical'}, 'floors': {'sdtype': 'categorical'}, 'waterfront': {'sdtype': 'categorical'}, 'view': {'sdtype': 'categorical'}, 'condition': {'sdtype': 'categorical'}, 'grade': {'sdtype': 'categorical'}, 'sqft_above': {'sdtype': 'numerical'}, 'sqft_basement': {'sdtype': 'numerical'}, 'yr_built': {'sdtype': 'numerical'}, 'yr_renovated': {'sdtype': 'numerical'}, 'zipcode': {'sdtype': 'categorical'}, 'lat': {'sdtype': 'numerical'}, 'long': {'sdtype': 'numerical'}, 'sqft_living15': {'sdtype': 'numerical'}, 'sqft_lot15': {'sdtype': 'numerical'}}, 'METADATA_SPEC_VERSION': 'SINGLE_TABLE_V1'}
Fitting Models
Params
{'metadata': {
    "columns": {
        "id": {
            "sdtype": "numerical"
        },
        "date": {
            "sdtype":

/usr/local/python/3.10.13/lib/python3.10/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(


{'columns': {'id': {'sdtype': 'numerical'}, 'date': {'sdtype': 'categorical'}, 'price': {'sdtype': 'numerical'}, 'bedrooms': {'sdtype': 'categorical'}, 'bathrooms': {'sdtype': 'categorical'}, 'sqft_living': {'sdtype': 'numerical'}, 'sqft_lot': {'sdtype': 'numerical'}, 'floors': {'sdtype': 'categorical'}, 'waterfront': {'sdtype': 'categorical'}, 'view': {'sdtype': 'categorical'}, 'condition': {'sdtype': 'categorical'}, 'grade': {'sdtype': 'categorical'}, 'sqft_above': {'sdtype': 'numerical'}, 'sqft_basement': {'sdtype': 'numerical'}, 'yr_built': {'sdtype': 'numerical'}, 'yr_renovated': {'sdtype': 'numerical'}, 'zipcode': {'sdtype': 'categorical'}, 'lat': {'sdtype': 'numerical'}, 'long': {'sdtype': 'numerical'}, 'sqft_living15': {'sdtype': 'numerical'}, 'sqft_lot15': {'sdtype': 'numerical'}}, 'METADATA_SPEC_VERSION': 'SINGLE_TABLE_V1'}
Fitting Models
Params
{'metadata': {
    "columns": {
        "id": {
            "sdtype": "numerical"
        },
        "date": {
            "sdtype":

/usr/local/python/3.10.13/lib/python3.10/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(


In [48]:
syns[0]

In [49]:
base_path = f"../docs/tesis/datasets/{DATASET_NAME.replace(' ','').lower()}-a"

In [50]:
def highlight_max_custom(series):
    # Extracción de los valores principales y los errores
    extracted = series.str.extract(r'([0-9.e+-]+)±([0-9.e+-]+)').astype(float)
    # Cálculo del valor modificado para el máximo
    modified_vals = extracted[0] 
    # Identificar el máximo
    max_val = modified_vals.max()
    return ['bfseries:;' if val == max_val else '' for val in modified_vals]

def highlight_min_custom(series):
    # Extracción de los valores principales y los errores
    extracted = series.str.extract(r'([0-9.e+-]+)±([0-9.e+-]+)').astype(float)
    # Cálculo del valor modificado para el mínimo
    modified_vals = extracted[0]
    # Identificar el mínimo
    min_val = modified_vals.min()
    return ['cellcolor:[rgb]{0.9, 0.54, 0.52};' if val == min_val else '' for val in modified_vals]


In [51]:
def scores_syns(syns):
    score_table = pd.concat([syn.scores.sort_values("score", ascending=False).assign(syni=i) for (i,syn) in enumerate(syns)]).groupby(["type", "name"]).agg({"score": lambda x: f"{np.mean(x):.2e}±{np.std(x):.2e}"}).reset_index().pivot(index="name", 
                                            columns=["type"],values="score").sort_values(
    "avg", ascending=False).rename(columns={'avg':'Score'}).loc[:,
    ["Column Pair Trends", "Column Shapes", "Coverage", "Boundaries", "Synthesis", "Score"]].reset_index().rename(columns={"name": "Model Name"}).rename(columns={"Score":"\\textbf{Score}"})
    formated_table = score_table.style.hide(axis="index").format(precision=3).format("\hline {}", score_table.columns[0], escape="latex").set_table_styles([
        {'selector': 'toprule', 'props': ':hline\n \\rowcolor[gray]{0.8};'},
        {'selector': 'bottomrule', 'props': ':hline;'}
    ], overwrite=False)\
    .apply(highlight_max_custom, subset=score_table.columns[1:])\
        .to_latex(
        column_format = f"|l|{'r|'*len(score_table.columns[1:])}",
        position="H",
        position_float="centering",
        caption = f"Evaluación de Métricas de Rendimiento para Diversos Modelos de Aprendizaje Automático, {DATASET_NAME.capitalize()}",
        label = f"table-score-{DATASET_NAME.lower()}-a",
        clines=None
    ).replace("\centering", "\\centering\n\\fontsize{7}{14}\\selectfont")
    with open(f"{base_path}/tables/table-score-{DATASET_NAME.lower()}-a-5th.tex", "w") as stext:
        stext.write(formated_table)
    return formated_table
scores_syns(syns)


'\\begin{table}[H]\n\\centering\n\\fontsize{7}{14}\\selectfont\n\\caption{Evaluación de Métricas de Rendimiento para Diversos Modelos de Aprendizaje Automático, King county}\n\\label{table-score-king county-a}\n\\begin{tabular}{|l|r|r|r|r|r|r|}\n\\hline\n \\rowcolor[gray]{0.8}\nModel Name & Column Pair Trends & Column Shapes & Coverage & Boundaries & Synthesis & \\textbf{Score} \\\\\n\\hline tddpm\\_mlp & \\bfseries 9.45e-01±2.59e-03 & \\bfseries 9.71e-01±3.78e-04 & \\bfseries 9.65e-01±4.55e-03 & \\bfseries 1.00e+00±0.00e+00 & \\bfseries 1.00e+00±0.00e+00 & \\bfseries 9.58e-01±1.35e-03 \\\\\n\\hline smote-enc & 9.41e-01±2.59e-04 & 9.65e-01±2.71e-04 & 8.40e-01±6.44e-03 & \\bfseries 1.00e+00±1.01e-05 & \\bfseries 1.00e+00±1.25e-04 & 9.53e-01±5.05e-05 \\\\\n\\hline tablepreset & 8.38e-01±0.00e+00 & 8.38e-01±6.41e-17 & 7.42e-01±0.00e+00 & \\bfseries 1.00e+00±0.00e+00 & \\bfseries 1.00e+00±0.00e+00 & 8.38e-01±1.57e-16 \\\\\n\\hline ctgan & 8.03e-01±2.69e-03 & 8.20e-01±1.37e-02 & 8.58e-01±1.

In [52]:
def coverage(syns):
    models = list(syns[0].get_details().keys())
    coverage_score = pd.concat([
        syn.get_details()[model]['diagnostic']['coverage'].assign(syni=i).assign(model=model)
        for (i, syn) in enumerate(syns)
        for model in ["smote-enc", "tddpm_mlp"]
    ]).groupby(["model", "Column", "Metric"]).agg({"Diagnostic Score": lambda x: f"{np.mean(x):.2e}±{np.std(x):.2e}"}).reset_index()\
    .pivot(index=["Column","Metric"], 
            values="Diagnostic Score", columns="model").sort_values("smote-enc", ascending=False).reset_index().rename(columns={"Column": "Columna", "Metric":"Metrica"})

    formated_coverage = coverage_score.sort_values("Columna").style.hide(axis="index")\
        .format("\hline {}", coverage_score.columns[0:1], escape="latex")\
        .format_index("{}", escape="latex", axis=1)\
        .set_table_styles([
        {'selector': 'toprule', 'props': ':hline\n\\rowcolor[gray]{0.8};'},
        {'selector': 'bottomrule', 'props': ':hline;'}
    ], overwrite=False)\
    .apply(highlight_max_custom, subset=coverage_score.columns[2:], axis=1)\
    .apply(highlight_min_custom, subset=coverage_score.columns[2:])\
        .to_latex(
        column_format = f"|l|l|{'r|'*len(coverage_score.columns[2:])}",
        position="H",
        position_float="centering",
        caption = f"Cobertura Categoría/Rango para Modelos Smote y Tddpm, {DATASET_NAME.capitalize()}",
        label = f"table-coverage-{DATASET_NAME.lower()}-a",
        clines=None
    )
    with open(f"{base_path}/tables/table-coverage-{DATASET_NAME.lower()}-a.tex", "w") as stext:
        stext.write(formated_coverage)
    return formated_coverage

print(coverage(syns))

\begin{table}[H]
\centering
\caption{Cobertura Categoría/Rango para Modelos Smote y Tddpm, King county}
\label{table-coverage-king county-a}
\begin{tabular}{|l|l|r|r|}
\hline
\rowcolor[gray]{0.8}
Columna & Metrica & smote-enc & tddpm\_mlp \\
\hline bathrooms & CategoryCoverage & 6.56e-01±1.57e-02 & \bfseries 8.33e-01±4.71e-02 \\
\hline bedrooms & CategoryCoverage & 5.64e-01±7.25e-02 & \bfseries \cellcolor[rgb]{0.9, 0.54, 0.52} 6.92e-01±6.28e-02 \\
\hline condition & CategoryCoverage & 8.00e-01±1.11e-16 & \bfseries 1.00e+00±0.00e+00 \\
\hline date & CategoryCoverage & \bfseries 9.70e-01±1.27e-03 & 9.36e-01±1.10e-02 \\
\hline floors & CategoryCoverage & 8.33e-01±0.00e+00 & \bfseries 1.00e+00±0.00e+00 \\
\hline grade & CategoryCoverage & 7.50e-01±0.00e+00 & \bfseries 8.33e-01±0.00e+00 \\
\hline id & RangeCoverage & 9.94e-01±2.27e-04 & \bfseries 1.00e+00±1.95e-04 \\
\hline lat & RangeCoverage & 9.67e-01±1.26e-02 & \bfseries 9.95e-01±6.95e-03 \\
\hline long & RangeCoverage & 9.91e-01±1.79e-

In [53]:
def union_mean_std(x):
    return f"{np.mean(x):.2e}±{np.std(x):.2e}"

PCT = {
    "5th": "percentil 5",
    "1th": "percentil 1",
    "min": "mínimo"
}

def shape(syns):
    models = list(syns[0].get_details().keys())
    coverage_score = pd.concat([
        syn.get_details()[model]['report']['column_shape'].assign(syni=i).assign(model=model)
        for (i, syn) in enumerate(syns)
        for model in ["smote-enc", "tddpm_mlp"]
    ]).groupby(["model", "Column", "Metric"]).agg({"Quality Score": union_mean_std}).reset_index()\
    .pivot(index=["Column","Metric"], 
            values="Quality Score", columns="model").sort_values("smote-enc", ascending=False).reset_index().rename(columns={"Column": "Columna", "Metric":"Metrica"})

    formated_coverage = coverage_score.sort_values("Columna").style.hide(axis="index")\
        .format("\hline {}", coverage_score.columns[0:1], escape="latex")\
        .format_index("{}", escape="latex", axis=1)\
        .set_table_styles([
        {'selector': 'toprule', 'props': ':hline\n\\rowcolor[gray]{0.8};'},
        {'selector': 'bottomrule', 'props': ':hline;'}
    ], overwrite=False)\
    .apply(highlight_max_custom, subset=coverage_score.columns[2:], axis=1)\
    .apply(highlight_min_custom, subset=coverage_score.columns[2:])\
        .to_latex(
        column_format = f"|l|l|{'r|'*len(coverage_score.columns[2:])}",
        position="H",
        position_float="centering",
        caption = f"Evaluación de Similitud de Distribución para Modelos Smote y Tddpm, {DATASET_NAME.capitalize()}",
        label = f"table-shape-{DATASET_NAME.lower()}-a",
        clines=None
    )
    with open(f"{base_path}/tables/table-shape-{DATASET_NAME.lower()}-a.tex", "w") as stext:
        stext.write(formated_coverage)
    return formated_coverage

print(shape(syns))

\begin{table}[H]
\centering
\caption{Evaluación de Similitud de Distribución para Modelos Smote y Tddpm, King county}
\label{table-shape-king county-a}
\begin{tabular}{|l|l|r|r|}
\hline
\rowcolor[gray]{0.8}
Columna & Metrica & smote-enc & tddpm\_mlp \\
\hline bathrooms & TVComplement & \cellcolor[rgb]{0.9, 0.54, 0.52} 8.84e-01±3.58e-03 & \bfseries 9.45e-01±3.05e-03 \\
\hline bedrooms & TVComplement & 9.20e-01±2.30e-03 & \bfseries 9.46e-01±2.27e-03 \\
\hline condition & TVComplement & 9.35e-01±3.21e-03 & \bfseries 9.61e-01±6.02e-04 \\
\hline date & TVComplement & \bfseries 9.38e-01±2.83e-03 & \cellcolor[rgb]{0.9, 0.54, 0.52} 9.28e-01±1.57e-03 \\
\hline floors & TVComplement & 9.65e-01±1.98e-03 & \bfseries 9.67e-01±1.24e-03 \\
\hline grade & TVComplement & 9.57e-01±4.16e-04 & \bfseries 9.63e-01±2.16e-03 \\
\hline id & KSComplement & \bfseries 9.87e-01±3.42e-04 & 9.80e-01±1.51e-03 \\
\hline lat & KSComplement & \bfseries 9.92e-01±6.06e-04 & 9.87e-01±1.68e-03 \\
\hline long & KSComplement 

In [54]:
def dcr_score(syns, pct="5th"):
    dcr_score = pd.concat([ syn.scores[syn.scores["type"] == "avg"].sort_values("score", ascending=False).loc[:,[f"DCR ST {pct}", f"DCR SH {pct}", f"DCR TH {pct}", "score"]]
    for syn in syns ]).sort_values("score", ascending=False).reset_index().groupby("name").agg(union_mean_std).loc[["tddpm_mlp", "smote-enc", "ctgan", "tablepreset", "copulagan", "gaussiancopula", "tvae"],:].reset_index().rename(columns={'name':"Modelo", "score": "\\textbf{Score}", f"DCR ST {pct}":"DCR ST", f"DCR SH {pct}": "DCR SH", f"DCR TH {pct}": "DCR TH" })

    formated_dcr = dcr_score.style.hide(axis="index")\
        .format(precision=3)\
        .format("\hline {}", dcr_score.columns[0], escape="latex")\
        .set_table_styles([
        {'selector': 'toprule', 'props': ':hline\n\\rowcolor[gray]{0.8};'},
        {'selector': 'bottomrule', 'props': ':hline;'}
    ], overwrite=False).apply(highlight_max_custom,
        subset=dcr_score.columns[1:],
        axis=0
    ).apply(highlight_min_custom,
        subset=dcr_score.columns[1:],
        axis=0
    ).to_latex(
        column_format = f"|l|l|{'r|'*len(dcr_score.columns[1:])}",
        position="H",
        position_float="centering",
        caption = f"Distancia de registros más cercanos entre conjuntos Sinteticos, {PCT[pct]}, {DATASET_NAME.capitalize()}",
        label = f"table-dcr-{DATASET_NAME.lower()}-a-{pct}",
        clines=None
    ).replace("\centering", "\\centering\n\\fontsize{10}{14}\\selectfont")
    with open(f"{base_path}/tables/table-dcr-{DATASET_NAME.lower()}-a-{pct}.tex", "w") as stext:
        stext.write(formated_dcr)
    return formated_dcr

print(dcr_score(syns))
print(dcr_score(syns, "1th"))
print(dcr_score(syns), "min")

\begin{table}[H]
\centering
\fontsize{10}{14}\selectfont
\caption{Distancia de registros más cercanos entre conjuntos Sinteticos, percentil 5, King county}
\label{table-dcr-king county-a-5th}
\begin{tabular}{|l|l|r|r|r|r|}
\hline
\rowcolor[gray]{0.8}
Modelo & DCR ST & DCR SH & DCR TH & \textbf{Score} \\
\hline tddpm\_mlp & 5.82e-02±6.47e-04 & 7.62e-02±5.61e-04 & \bfseries \cellcolor[rgb]{0.9, 0.54, 0.52} 3.57e-02±0.00e+00 & \bfseries 9.58e-01±1.35e-03 \\
\hline smote-enc & \cellcolor[rgb]{0.9, 0.54, 0.52} 7.12e-03±2.73e-04 & \cellcolor[rgb]{0.9, 0.54, 0.52} 3.72e-02±4.54e-04 & \bfseries \cellcolor[rgb]{0.9, 0.54, 0.52} 3.57e-02±0.00e+00 & 9.53e-01±5.05e-05 \\
\hline ctgan & 2.21e-01±3.48e-03 & 2.45e-01±4.59e-03 & \bfseries \cellcolor[rgb]{0.9, 0.54, 0.52} 3.57e-02±0.00e+00 & 8.12e-01±7.96e-03 \\
\hline tablepreset & 1.78e-01±0.00e+00 & 1.97e-01±0.00e+00 & \bfseries \cellcolor[rgb]{0.9, 0.54, 0.52} 3.57e-02±0.00e+00 & 8.38e-01±9.06e-17 \\
\hline copulagan & \bfseries 3.71e-01±1.23e-02 &

In [55]:
def nndr_score(syns, pct="5th"):
    dcr_score = pd.concat([ syn.scores[syn.scores["type"] == "avg"].sort_values("score", ascending=False).loc[:,[f"NNDR ST {pct}", f"NNDR SH {pct}", f"NNDR TH {pct}", "score"]]
    for syn in syns ]).reset_index().sort_values("score", ascending=False).groupby("name").agg(union_mean_std).loc[["tddpm_mlp", "smote-enc", "ctgan", "tablepreset", "copulagan", "gaussiancopula", "tvae"],:].reset_index().rename(columns={'name':"Modelo", "score": "\\textbf{Score}", f"NNDR ST {pct}": "NNDR ST", f"NNDR SH {pct}": "NNDR SH", f"NNDR TH {pct}": "NNDR TH"})

    

    formated_dcr = dcr_score.style.hide(axis="index")\
        .format(precision=3)\
        .format("\hline {}", dcr_score.columns[0], escape="latex")\
        .set_table_styles([
        {'selector': 'toprule', 'props': ':hline\n\\rowcolor[gray]{0.8};'},
        {'selector': 'bottomrule', 'props': ':hline;'}
    ], overwrite=False).apply(highlight_max_custom,
        subset=dcr_score.columns[1:],
        axis=0
    ).apply(highlight_min_custom,
        subset=dcr_score.columns[1:],
        axis=0
    ).to_latex(
        column_format = f"|l|l|{'r|'*len(dcr_score.columns[1:])}",
        position="H",
        position_float="centering",
        caption = f"Proporción entre el más cercano y el segundo más cercano, {PCT[pct]}, {DATASET_NAME.capitalize()}",
        label = f"table-nndr-{DATASET_NAME.lower()}-a-{pct}",
        clines=None
    ).replace("\centering", "\\centering\n\\fontsize{10}{14}\\selectfont")
    with open(f"{base_path}/tables/table-nndr-{DATASET_NAME.lower()}-a-{pct}.tex", "w") as stext:
        stext.write(formated_dcr)
    return formated_dcr

print(nndr_score(syns))
print(nndr_score(syns, "1th"))
print(nndr_score(syns, "min"))

\begin{table}[H]
\centering
\fontsize{10}{14}\selectfont
\caption{Proporción entre el más cercano y el segundo más cercano, percentil 5, King county}
\label{table-nndr-king county-a-5th}
\begin{tabular}{|l|l|r|r|r|r|}
\hline
\rowcolor[gray]{0.8}
Modelo & NNDR ST & NNDR SH & NNDR TH & \textbf{Score} \\
\hline tddpm\_mlp & 6.12e-01±3.23e-03 & 6.05e-01±4.01e-03 & \bfseries \cellcolor[rgb]{0.9, 0.54, 0.52} 3.76e-01±0.00e+00 & \bfseries 9.58e-01±1.35e-03 \\
\hline smote-enc & \cellcolor[rgb]{0.9, 0.54, 0.52} 1.92e-01±7.02e-03 & \cellcolor[rgb]{0.9, 0.54, 0.52} 4.19e-01±4.47e-03 & \bfseries \cellcolor[rgb]{0.9, 0.54, 0.52} 3.76e-01±0.00e+00 & 9.53e-01±5.05e-05 \\
\hline ctgan & 8.10e-01±1.98e-03 & 8.16e-01±4.08e-03 & \bfseries \cellcolor[rgb]{0.9, 0.54, 0.52} 3.76e-01±0.00e+00 & 8.12e-01±7.96e-03 \\
\hline tablepreset & \bfseries 8.29e-01±6.41e-17 & 8.13e-01±0.00e+00 & \bfseries \cellcolor[rgb]{0.9, 0.54, 0.52} 3.76e-01±0.00e+00 & 8.38e-01±9.06e-17 \\
\hline copulagan & 8.20e-01±3.87e-03 & \